# Kaggle Miniproject 1

## Team Members:
### Karthik Karnik 
### Kapil Sinha
### Anvita Mishra

In [1]:
import numpy as np

In [9]:
with open("training_data.txt") as f:
    words = f.readline().split(' ')
training_data = np.loadtxt("training_data.txt", skiprows = 1)
test_data = np.loadtxt("test_data.txt", skiprows = 1)

In [12]:
x_train = training_data[1:]
y_train = training_data[0]